<a href="https://colab.research.google.com/github/SonjaWangJQ/Keyboard-ML/blob/main/3_FFT_on_claasifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LOAD DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:

from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle

import math, random
import torch
import torchaudio
import librosa
from torchaudio import transforms
from IPython.display import Audio
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, random_split
import csv
import matplotlib.pyplot as plt

In [ ]:


class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------

  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)



    # ----------------------------
  # 数据扩充--时间移动（1）
  #7
  @staticmethod
  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)



  @staticmethod
  def compute_mfcc(aud, n_mfcc=32, window_size=0.01, step_size=0.0025, n_filters=32):

            sig, sr = aud
            n_fft = int(sr * window_size)
            hop_length = int(sr * step_size)

            mfccs = librosa.feature.mfcc(
                y=sig.numpy(),
                sr=sr,
                n_mfcc=n_mfcc,
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=n_filters
            )
            return mfccs.T


  @staticmethod
  def compute_stft(aud, n_fft=2048, hop_length=512, window='hann'):
    sig, sr = aud
    audio = sig.squeeze().numpy()

    # Compute STFT
    stft_result = librosa.core.stft(audio, n_fft=2048, hop_length=512, window='hann')

    # Get magnitude spectrum
    magnitude = np.abs(stft_result)

    # Get frequency axis
    freqs = librosa.core.fft_frequencies(sr=sr, n_fft=2048)

    return magnitude


In [ ]:

# ----------------------------
# SoundDS +10ms
# ----------------------------
class SoundDS1(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    # self.train_indices = train_indices  # 训练集的索引列表
    self.sr = 44100
    self.channel = 1
    self.shift_pct = 0.01   # +10ms
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts





  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]


    aud = AudioUtil.open(audio_path)
    shift_aud = AudioUtil.time_shift(aud, self.shift_pct)
    # mfcc_aud = AudioUtil.compute_mfcc(shift_aud)
    # mfcc_data = np.array(mfcc_aud)
    fft_audio = AudioUtil.compute_stft(shift_aud )
    fft_data = np.array(fft_audio)

    return fft_audio, class_id, audio_path


In [ ]:

# ----------------------------
# SoundDS2 -10ms
# ----------------------------
class SoundDS2(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    # self.train_indices = train_indices # 训练集的索引列表
    self.sr = 44100
    self.channel = 1
    self.shift_pct = -0.01   # +10ms
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)

  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts




  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]

    aud = AudioUtil.open(audio_path)
    shift_aud = AudioUtil.time_shift(aud, self.shift_pct)
    # mfcc_aud = AudioUtil.compute_mfcc(shift_aud)
    # mfcc_data = np.array(mfcc_aud)
    fft_audio = AudioUtil.compute_stft(shift_aud )
    fft_data = np.array(fft_audio)

    return fft_data, class_id, audio_path


In [ ]:

# ----------------------------
# SoundDS1 no shift
# ----------------------------
class SoundDS(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    self.sr = 44100
    self.channel = 1
    # self.shift_pct = -0.01   # +10ms
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts


  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]

    aud = AudioUtil.open(audio_path)
    # shift_aud = AudioUtil.time_shift(aud, self.shift_pct)
    # mfcc_aud = AudioUtil.compute_mfcc(aud)
    # mfcc_data = np.array(mfcc_aud)
    fft_audio = AudioUtil.compute_stft(aud)
    fft_data = np.array(fft_audio)

    return fft_data, class_id, audio_path # 为了创建映射




---



In [ ]:
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset= SoundDS(csv_filename)
# dataloader = DataLoader(dataset, batch_size=5, shuffle=True)
X = []
y = []
path = []
for idx in range(len(sound_dataset)):
    fft_data, class_id, audio_path = sound_dataset[idx]  # 调用 __getitem__ 方法
    X.append(fft_data)
    y.append(class_id)
    path.append(audio_path)



In [ ]:
X = np.array(X)
y = np.array(y)
path = np.array(path)
print(X.shape,y.shape)
# 检查 X 的行数与 y 的长度是否一致
assert X.shape[0] == len(y), "Shape mismatch between X and y"
assert X.shape[0] == len(path), "Shape mismatch between X and path"
# X_train,y 0.2
X_input = X.reshape(X.shape[0], -1)  # Reshape to (1173, 1025*9)

X_train, X_test, y_train, y_test,path_train, path_test = train_test_split(X_input, y,path, test_size=0.2, random_state=42, stratify=y)



(1467, 1025, 9) (1467,)


In [ ]:
print(X_train.shape, X_test.shape, y_test.shape,path_train.shape)

(1173, 9225) (294, 9225) (294,) (1173,)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape, X_test_scaled.shape)

(1173, 9225) (294, 9225)


这时我们得到了train data路径, path-train(0.2*1407,)



---



### APPEND NEW

In [ ]:
# shift -10 append into X,y
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift2= SoundDS1(csv_filename)
X2 = []
y2 = []

for idx in range(len(sound_dataset_shift2)):
    fft_data2, class_id2, audio_path2 = sound_dataset_shift2[idx]  # 调用 __getitem__ 方法
    if audio_path2 in path_train:
        X2.append(fft_data2)
        y2.append(class_id2)


In [ ]:

csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift3= SoundDS2(csv_filename)

for idx in range(len(sound_dataset_shift3)):
    fft_data3, class_id3, audio_path3 = sound_dataset_shift3[idx]  # 调用 __getitem__ 方法
    if audio_path3 in path_train:
        X2.append(fft_data3)
        y2.append(class_id3)

In [ ]:
X2 = np.array(X2)
y2 = np.array(y2)
# 检查 X 的行数与 y 的长度是否一致
assert X2.shape[0] == len(y2), "Shape mismatch between X and y"
num_samples, num_frames, num_coefficients, num_channels = X2.shape

new_X2train = X2.reshape(num_samples, num_frames * num_coefficients)
print(new_X2train.shape)

(2346, 1312)


# new append sample number

In [ ]:

X2_shuffled, y2_shuffled = shuffle( new_X2train , y2, random_state=42)
new_X2_train_scaled = scaler.fit_transform(X2_shuffled)
print(new_X2_train_scaled.shape, y2_shuffled)


(2346, 1312) [11  5 15 ... 34  6 17]


In [ ]:
# append X_train, y_train
X_train_extended = np.vstack([X_train_scaled, new_X2_train_scaled])
y_train_extended = np.concatenate([y_train, y2_shuffled])

In [ ]:
num_classes = len(np.unique(y_train_extended)) # 或者使用 len(np.unique(y_test))
print(y_train_extended)
print("Number of classes:", num_classes)

[20 12 32 ... 34  6 17]
Number of classes: 36


In [ ]:
print(X_train_extended.shape,y_train_extended.shape)
print(X_test_scaled.shape, y_test.shape)
# print(X_val_scaled.shape,X_test_scaled.shape, y_test.shape,y_valid.shape,)

(3519, 1312) (3519,)
(294, 1312) (294,)


**start training**

# SVM model 使用原数据训练结果 fft_SVM数据shift

In [ ]:
#SVM model 使用原数据训练结果
#MFCC_SVM数据shift
# Best parameters: {'C': 10, 'kernel': 'rbf'} 0.84
#Mfcc: aud, n_mfcc=26, window_size=0.01, step_size=0.0025, n_filters=64
#  parameter grid for grid search
from sklearn.metrics import top_k_accuracy_score


param_grid = {
    'C': [1, 5, 10, 100],
    # 'C': [10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001]
    # 'gamma':[0.001]
    }

#  SVM model
svm = SVC()

#  grid search object with 10-fold cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=10, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

#  best model from grid search
best_model = grid_search.best_estimator_

# Predict using the best model
y_pred = best_model.predict(X_test_scaled)
target_names = [f'Class {i}' for i in range(36)]
#   classification report
cr = classification_report(y_test, y_pred, target_names=target_names)
# N = 5  # Change this to the desired value
# top_n_accuracy = top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N)



N_values = [1, 2, 3, 4, 5]
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N) for N in N_values]

# Print results
print("Best parameters:", grid_search.best_params_)
print("Classification Report:")
print(cr)
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.62      0.62      0.62         8
     Class 1       1.00      0.67      0.80         9
     Class 2       0.60      0.33      0.43         9
     Class 3       0.50      0.60      0.55        10
     Class 4       0.31      0.89      0.46         9
     Class 5       0.67      0.25      0.36         8
     Class 6       0.58      0.78      0.67         9
     Class 7       0.73      0.80      0.76        10
     Class 8       0.50      0.80      0.62        10
     Class 9       0.50      0.60      0.55        10
    Class 10       0.83      0.71      0.77         7
    Class 11       0.38      0.33      0.35         9
    Class 12       1.00      0.50      0.67         8
    Class 13       0.67      0.29      0.40         7
    Class 14       0.75      0.33      0.46         9
    Class 15       1.00      0.56      0.71     

In [ ]:
from matplotlib.ticker import PercentFormatter
# Plotting the top-N accuracy scores
plt.figure(figsize=(8, 6))
plt.plot(N_values, top_n_accuracies, marker='o')
plt.ylim(0, 1)  # Set y-axis limits between 0 and 1
plt.xticks(N_values)
plt.xlabel("Number of guesses")
plt.ylabel("Accuracy")
# plt.title("")
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1)) # %
plt.grid(True)
plt.show()

### data shift +- 10ms

In [ ]:
#SVM model
# data shift
# X_train_extended.shape,y_train_extended.shape
# Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
from sklearn.metrics import top_k_accuracy_score


param_grid = {
    'C': [1],
    'kernel': ['linear'],
    # 'gamma': ['scale', 'auto', 0.1, 0.01, 0.001]
    'gamma':["scale"]
    }

#  SVM model
svm = SVC()

#  grid search object with 10-fold cross-validation
# grid_search = GridSearchCV(svm, param_grid, cv=10, n_jobs=-1)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=10, n_jobs=-1) # 新修改
grid_search.fit(X_train_extended, y_train_extended)
# grid_search.fit(X_train_scaled, y_train)

#  best model from grid search
best_model = grid_search.best_estimator_

# Predict using the best model
y_pred = best_model.predict(X_test_scaled)
target_names = [f'Class {i}' for i in range(36)]
#   classification report
cr = classification_report(y_test, y_pred, target_names=target_names)
# TOP 5 ACCURACY
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N) for N in N_values]

# Print results
print("parameters:", grid_search.best_params_)
print("Classification Report:")
print(cr)
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

In [ ]:
from matplotlib.ticker import PercentFormatter
# Plotting the top-N accuracy scores
plt.figure(figsize=(8, 6))
plt.plot(N_values, top_n_accuracies, marker='o')
plt.ylim(0, 1)  # Set y-axis limits between 0 and 1
plt.xticks(N_values)
plt.xlabel("Number of guesses")
plt.ylabel("Accuracy")
# plt.title("")
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1)) # %
plt.grid(True)
plt.show()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
#'C': [0.7, 1, 2, 5] 正则化
# https://zhuanlan.zhihu.com/p/409250935

In [ ]:
# MLR


### Multinomial Logistic Regression

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_digits
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
# data shift 数据 origial

# Multinomial Logistic Regression  X,y 0.2 ；L2；10 fold across MFCC

#X,y 0.2 ；L2；10 fold across MFCC

multi_logreg = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter=1000)

param_grid = {
    'C': [ 1,10 ],
    'penalty': ['l2']
}

# 10 fold across
grid_search = GridSearchCV(multi_logreg, param_grid, cv=10, n_jobs=-1)

grid_search.fit(X_train_scaled, y_train)
print("Best parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)



target_names = [f'Class {i}' for i in range(36)]

# report
cr = classification_report(y_test, y_pred, target_names=target_names)

# Top-N accuracy score
N_values = [1, 2, 3, 4, 5]  # You can add more N values if needed
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.predict_proba(X_test_scaled), k=N) for N in N_values]

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Print classification report
print("Classification Report:")
print(cr)

Best parameters: {'C': 10, 'penalty': 'l2'}
Top-1 Accuracy Score: 0.5748
Top-2 Accuracy Score: 0.6633
Top-3 Accuracy Score: 0.7211
Top-4 Accuracy Score: 0.7653
Top-5 Accuracy Score: 0.7959
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.43      0.75      0.55         8
     Class 1       1.00      0.67      0.80         9
     Class 2       0.60      0.33      0.43         9
     Class 3       0.67      0.60      0.63        10
     Class 4       0.43      0.67      0.52         9
     Class 5       0.80      0.50      0.62         8
     Class 6       0.50      0.67      0.57         9
     Class 7       0.73      0.80      0.76        10
     Class 8       0.60      0.90      0.72        10
     Class 9       0.45      0.50      0.48        10
    Class 10       0.40      0.57      0.47         7
    Class 11       0.50      0.56      0.53         9
    Class 12       0.67      0.50      0.57         8
    Class 13       0.60      0.

SHIFT DATA *3

In [ ]:
multi_logreg = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter=1000)

# parameters: {'C': 1, 'penalty': 'l2'}
param_grid = {
    'C': [ 1 ],
    'penalty': ['l2']
}

# 10 fold across
grid_search = GridSearchCV(multi_logreg, param_grid, cv=10, n_jobs=-1)

grid_search.fit(X_train_extended, y_train_extended) # *3
print("parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)



target_names = [f'Class {i}' for i in range(36)]

# report
cr = classification_report(y_test, y_pred, target_names=target_names)

# Top-N accuracy score
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.predict_proba(X_test_scaled), k=N) for N in N_values]

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Print classification report
print("Classification Report:")
print(cr)

### random forest

In [ ]:
import pydot
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import classification_report

# Assuming you have loaded X_train_scaled, y_train, X_test_scaled, and y_test

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [ 5, 10],
    'min_samples_leaf': [ 2, 4],
    'bootstrap': [True, False]
}

# Create and fit the Random Forest model using GridSearchCV
random_forest = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(random_forest, param_grid, cv=10, n_jobs=-1) # 这个cv是什么？
grid_search.fit(X_train_scaled, y_train)

best_random_forest = grid_search.best_estimator_

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
y_test_pred_probs = cross_val_predict(best_random_forest, X_test_scaled, y_test, cv=kf, method='predict_proba')

#  top-N accuracy scores and generating the classification report
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = []
for N in N_values:
    top_n_correct = 0
    for true_label, pred_probs in zip(y_test, y_test_pred_probs):
        top_n_indices = np.argsort(pred_probs)[::-1][:N]  # Indices of top-N classes
        if true_label in top_n_indices:
            top_n_correct += 1
    top_n_accuracy = top_n_correct / len(y_test)
    top_n_accuracies.append(top_n_accuracy)

# Print the best hyperparameters found by GridSearchCV
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Generate classification report
target_names = [f'Class {i}' for i in range(36)]
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
cr = classification_report(y_test, y_test_pred, target_names=target_names)
# Print classification report
print("Classification Report:")
print(cr)



Best Hyperparameters:
{'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 100}
Top-1 Accuracy Score: 0.2279
Top-2 Accuracy Score: 0.3061
Top-3 Accuracy Score: 0.3810
Top-4 Accuracy Score: 0.4252
Top-5 Accuracy Score: 0.4660
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.12      0.12      0.12         8
     Class 1       0.18      0.22      0.20         9
     Class 2       0.23      0.33      0.27         9
     Class 3       0.29      0.50      0.37        10
     Class 4       0.40      0.22      0.29         9
     Class 5       0.00      0.00      0.00         8
     Class 6       0.58      0.78      0.67         9
     Class 7       0.12      0.20      0.15        10
     Class 8       0.26      0.50      0.34        10
     Class 9       0.38      0.50      0.43        10
    Class 10       0.00      0.00      0.00         7
    Class 11       0.29      0.22      0.25         9

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
